In [3]:
import numpy as np
import tifffile
import cv2
import json
import os
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ImageProcessor:
    def __init__(self, model_path='../Pipeline/model_efficientNetB7.pt'):
        # Load the cell count model
        
        NN = models.efficientnet_b7(weights="IMAGENET1K_V1")

        #resnet not trained later 
        for param in NN.parameters():
            param.requires_grad = False

        num_features = NN.classifier[-1].in_features

        mlp = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 1),
            #nn.Softmax(dim=1)
        )

        NN.classifier = mlp

        NN = NN.to(device)
        
        self.segmentation_and_regression_model = NN
        state_dict = torch.load(model_path, map_location = device)
        self.segmentation_and_regression_model.load_state_dict(state_dict)
        self.segmentation_and_regression_model.eval()
       

    def process_image(self, image_path):
        # Load the image from the .tif file
        image = tifffile.imread(image_path)

        # Apply your defined function to separate slices
        droplet_segments, x_coordinates = self.identify_droplet_segments(image)

        # Initialize an empty list to store cell counts and x coordinates
        results = []

        # Iterate over slices and pass through the pretrained neural network
        for i, slice in enumerate(droplet_segments):
            cell_count = self.nbCells(frame = slice, display= False)
            # Append results to the list
            results.append({"x_coordinate": x_coordinates[i], "cell_count": cell_count})

        return results
    
    def nbCells(self, frame, display = False):
        frame = frame.astype(np.uint8)
        magnitude = self.sobel_filter(frame)
        
        mask = (magnitude > 150).astype(np.uint8) * 255
        
        circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, 1, 20, param1=1000, param2=10, minRadius=10, maxRadius=20)
        
        if display:
            circles = np.uint16(np.around(circles))

            img_with_circles = np.copy(frame)

            for c in circles[0, :]:
                # Draw the outer circle
                cv2.circle(img_with_circles, (c[0], c[1]), c[2], (0, 255, 0), 3)
                # Draw the center of the circle
                cv2.circle(img_with_circles, (c[0], c[1]), 1, (0, 0, 255), 5)
        
        return circles.shape[1]

    def identify_droplet_segments(self, image):

        # 1. Convert the image to greyscale
        grey_image = image[:, :, 0]

        # 2. Apply filtering
        sobel_filtered_image = self.sobel_filter(grey_image)

        # 3. Apply slice on x to get a list of the slices
        segments_x = self.slice_along_x(sobel_filtered_image)

        # setup segments
        image_segments = []
        mean_x_value = []

        for segment_x in segments_x:
            coordinates_y = self.slice_along_y(image = sobel_filtered_image, coordinates= segment_x)
            original_segment = grey_image[coordinates_y[0][0]:coordinates_y[0][1],segment_x[0]:segment_x[1]]
            original_segment = original_segment.astype(int)
            #self.visualize_image(original_segment)
            image_segments.append(original_segment)
            mean_x_value.append(np.mean(segment_x))

        return image_segments, mean_x_value

    def visualize_image(self, image):
        plt.imshow(image, cmap='gray')
        plt.show()

    def sobel_filter(self, gray_arr):
        # Apply Sobel filter to the image
        sobel_x = cv2.Sobel(gray_arr, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray_arr, cv2.CV_64F, 0, 1, ksize=3)
        mask = np.sqrt(sobel_x**2 + sobel_y**2)
        return mask

    def slice_along_x(self, image, threshold_start=3000, threshold_end=2000, min_segment_length=400, max_segment_length=1500):
        # Calculate variance along the x-axis
        x_variances = np.var(image, axis=0)

        #plt.plot(x_variances)
        #plt.title('Variance along the x-axis')
        #plt.xlabel('X-axis')
        #plt.ylabel('Variance')
        #plt.show()

        # Identify segments where the variance exceeds the threshold
        segments = []
        droplet_started = False
        start_index = 0

        for i, value in enumerate(x_variances):
            if value > threshold_start and not droplet_started:
                droplet_started = True
                start_index = i
            elif value <= threshold_end and droplet_started:
                droplet_started = False
                end_index = i - 1
                segment_length = end_index - start_index

                # Check if the segment length is within the desired range
                if min_segment_length <= segment_length <= max_segment_length:
                    segments.append((start_index, end_index))

        # If a droplet continues to the end of the image, consider it
        if droplet_started:
            end_index = len(x_variances) - 1
            segment_length = end_index - start_index

            # Check if the segment length is within the desired range
            if min_segment_length <= segment_length <= max_segment_length:
                segments.append((start_index, end_index))

        # Extract slices based on the identified positions
        coordinates = [(start, end) for start, end in segments]

        return coordinates

    def slice_along_y(self, image, coordinates, y_threshold_start=3000, y_threshold_end=500,
                      min_segment_length=400, max_segment_length=800):
        # Calculate variance along the x-axis
        image = image[:,coordinates[0]:coordinates[1]]
        y_variances = np.var(image, axis=1)

        #plt.plot(y_variances)
        #plt.title('Variance along the y-axis')
        #plt.xlabel('Y-axis')
        #plt.ylabel('Variance')
        #plt.show()

        # Identify segments where the variance exceeds the threshold
        segments = []
        droplet_started = False
        start_index = 0

        for i, value in enumerate(y_variances):
            if value > y_threshold_start and not droplet_started:
                droplet_started = True
                start_index = i
            elif value <= y_threshold_end and droplet_started:
                droplet_started = False
                end_index = i - 1
                segment_length = end_index - start_index

                # Check if the segment length is within the desired range
                if min_segment_length <= segment_length <= max_segment_length:
                    segments.append((start_index, end_index))

        # Extract slices based on the identified positions
        coordinates_y = [(start, end) for start, end in segments]

        return coordinates_y
    
if __name__ == "__main__":
    inputPath = "../Data/20231206160503Z_test/frames"
    outputPath = "../Data"
    
    # Initialize lists to store results for each frame
    all_results = []
    droplet_segment_tensors_list = []

    # Iterate through each file in inputPath
    for filename in os.listdir(inputPath):
        if filename.endswith(".tif"):
            image_path = os.path.join(inputPath, filename)

            # Process each image
            image_processor = ImageProcessor()
            results = image_processor.process_image(image_path)

            # Append results to the all_results list
            all_results.append({filename: results})

    # Create JSON structure with frames sorted numerically
    json_output = {}
    for result in sorted(all_results, key=lambda x: int(list(x.keys())[0].split(".")[0])):
        try:
            filename_key = list(result.keys())[0]
            frame_number = int(filename_key.split(".")[0])
            
            # Check if the frame_number key exists in json_output, if not, create an empty list
            if frame_number not in json_output:
                json_output[frame_number] = []

            # Iterate over each entry in the result and append to the frame_number key
            for entry in result[filename_key]:
                json_output[frame_number].append([entry["x_coordinate"], entry["cell_count"]])

        except IndexError as e:
            print(f"Error processing result: {result}")
            print(f"Error details: {e}")

    # Sort the dictionary by keys
    json_output = dict(sorted(json_output.items()))

    # Write JSON to file
    output_file_path = os.path.join(outputPath, "output.json")
    with open(output_file_path, 'w') as json_file:
        json.dump(json_output, json_file, indent=2)

    print(f"JSON output written to {output_file_path}")


TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offs

[{'1.tif': [{'x_coordinate': 279.0, 'cell_count': 49}, {'x_coordinate': 908.0, 'cell_count': 36}, {'x_coordinate': 1471.5, 'cell_count': 34}]}, {'10.tif': [{'x_coordinate': 455.5, 'cell_count': 53}, {'x_coordinate': 1079.0, 'cell_count': 57}, {'x_coordinate': 1709.5, 'cell_count': 42}]}, {'2.tif': [{'x_coordinate': 562.0, 'cell_count': 41}, {'x_coordinate': 1216.5, 'cell_count': 35}]}, {'3.tif': [{'x_coordinate': 278.5, 'cell_count': 39}, {'x_coordinate': 869.5, 'cell_count': 55}]}, {'4.tif': [{'x_coordinate': 866.0, 'cell_count': 98}]}, {'5.tif': [{'x_coordinate': 273.0, 'cell_count': 55}, {'x_coordinate': 858.5, 'cell_count': 35}, {'x_coordinate': 1485.5, 'cell_count': 50}]}, {'6.tif': [{'x_coordinate': 543.0, 'cell_count': 62}, {'x_coordinate': 1153.5, 'cell_count': 36}, {'x_coordinate': 1696.0, 'cell_count': 38}]}, {'7.tif': [{'x_coordinate': 249.0, 'cell_count': 54}, {'x_coordinate': 843.5, 'cell_count': 59}, {'x_coordinate': 1419.0, 'cell_count': 31}]}, {'8.tif': [{'x_coordinate'

IndexError: list index out of range